# UMC 301: Applied Data Science and Artificial Intelligence
## Assignment 3

### Submission instructions:

1.   The assignment is to be submitted in ONE single notebook.
2.   Submit the .ipynb file with all cells open and the pdf for part 2 through Teams Assignment.
3. If your IISc email ID is < username > @iisc.ac.in, then name the file < username >_Assgn_3. E.g. jonathan_Assgn_3 for email ID jonathan@iisc.ac.in.
4. Before submission, execute the ’Restart session and run all’ option from the Runtime/Kernel tab. Verify that there are no errors and that you are getting the output you expect.
5. Use the dataset: https://github.com/taivop/joke-dataset
6. The assignment is divided into two questions.

- Dropping data
- num_epochs ine xperiment tracker

## Question 1

### Part 1

Load the dataset, create train, validation and test splits. Preprocess the dataset as required to train a decoder model. (10 marks)

In [1]:
!git clone https://github.com/taivop/joke-dataset # clone the dataset

Cloning into 'joke-dataset'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 44 (delta 12), reused 10 (delta 10), pack-reused 30 (from 1)
Receiving objects: 100% (44/44), 32.38 MiB | 14.11 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [2]:
import json
import csv
import os
import pandas as pd
import re
import torch
from torch.nn import CrossEntropyLoss
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
import math
import time
import torch.optim as optim
import torch.nn.functional as F
import datetime
from datetime import datetime
import random
import matplotlib.pyplot as plt
import time
import shutil
import warnings
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import Dataset
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import Dataset
import logging
from pathlib import Path
from typing import Dict, Optional, Union
from torch.optim.lr_scheduler import ReduceLROnPlateau
from typing import Dict, Any, List
from contextlib import nullcontext
from fpdf import FPDF
import sys
import io
from contextlib import redirect_stdout
import gc
import itertools
import json

/home/pratyushkant/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extract the ASCII characters (a to z, A to Z, 0 to 9) from the body category of the JSON files (stupidstuff.json, wocka.json) and from both title and body for reddit_jokes.json and save it as a csv file in another directory 'output_csv'.

In [3]:
# Function to extract ASCII characters
def extract_ascii(text):
    # Use regex to extract only alphanumeric characters (a-z, A-Z, 0-9) and spaces
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

# Function to write extracted text to CSV
def write_to_csv(data, file_path, headers):
    with open(file_path, mode='w', newline='', encoding='utf-8') as file: # Open the file in write mode
        writer = csv.writer(file) # Create a CSV writer
        writer.writerow(headers) # Write the headers
        for row in data: # Iterate over the data
            writer.writerow(row) # Write the row

# Process the stupidstuff.json and wocka.json files
def process_single_body_json(file_path, output_csv):
    extracted_data = [] # Initialize an empty list to store the extracted data

    # Load JSON file
    with open(file_path, 'r', encoding='utf-8') as file:
        jokes = json.load(file) # Load the JSON file

    # Extract ASCII from the 'body' field of each joke
    for joke in jokes:  # Iterate over the jokes
        if 'body' in joke: # Check if the joke has a 'body' field
            clean_body = extract_ascii(joke['body']) # Extract ASCII characters from the 'body' field
            extracted_data.append([clean_body]) # Append the extracted text to the list

    # Write to CSV
    write_to_csv(extracted_data, output_csv, headers=['body'])

# Process the reddit_jokes.json file
def process_reddit_jokes_json(file_path, output_csv):
    extracted_data = [] # Initialize an empty list to store the extracted data

    # Load JSON file
    with open(file_path, 'r', encoding='utf-8') as file:
        jokes = json.load(file) # Load the JSON file

    # Extract ASCII from both 'title' and 'body', concatenate them
    for joke in jokes: # Iterate over the jokes
        if 'title' in joke and 'body' in joke: # Check if the joke has both 'title' and 'body' fields
            clean_title = extract_ascii(joke['title']) # Extract ASCII characters from the 'title' field
            clean_body = extract_ascii(joke['body']) # Extract ASCII characters from the 'body' field
            concatenated = clean_title + " " + clean_body # Concatenate the title and body
            extracted_data.append([concatenated]) # Append the extracted text to the list

    # Write to CSV
    write_to_csv(extracted_data, output_csv, headers=['body']) # Write the extracted data to a CSV file

# Paths for input and output files
input_folder = 'joke-dataset' # Folder containing the JSON files
output_folder = 'output_csv' # Folder to store the CSV files

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True) # Create the output folder if it does not exist

# Process stupidstuff.json
process_single_body_json(os.path.join(input_folder, 'stupidstuff.json'), os.path.join(output_folder, 'stupidstuff.csv')) # Process the stupidstuff.json file

# Process wocka.json
process_single_body_json(os.path.join(input_folder, 'wocka.json'), os.path.join(output_folder, 'wocka.csv')) # Process the wocka.json file

# Process reddit_jokes.json
process_reddit_jokes_json(os.path.join(input_folder, 'reddit_jokes.json'), os.path.join(output_folder, 'reddit_jokes.csv')) # Process the reddit_jokes.json file

print("CSV files have been successfully generated.") # Print a message to indicate that the CSV files have been generated

CSV files have been successfully generated.


In [4]:
# Report all the columns for each CSV file

# Load the CSV files
stupidstuff_df = pd.read_csv(os.path.join(output_folder, 'stupidstuff.csv'))
wocka_df = pd.read_csv(os.path.join(output_folder, 'wocka.csv'))
reddit_jokes_df = pd.read_csv(os.path.join(output_folder, 'reddit_jokes.csv'))

# Display the columns for each CSV file
print("stupidstuff.csv columns:")
print(stupidstuff_df.columns)
print("\nwocka.csv columns:")
print(wocka_df.columns)
print("\nreddit_jokes.csv columns:")
print(reddit_jokes_df.columns)

stupidstuff.csv columns:
Index(['body'], dtype='object')

wocka.csv columns:
Index(['body'], dtype='object')

reddit_jokes.csv columns:
Index(['body'], dtype='object')


Merge all the csv files as joke.csv in the same directory as this notebook.

In [5]:
# Merge the three dataframes into a single dataframe
# named all_jokes_df and save it as a CSV file

all_jokes_df = pd.concat([stupidstuff_df, wocka_df, reddit_jokes_df], ignore_index=True)
all_jokes_df.to_csv(os.path.join('all_jokes.csv'), index=False)

print("all_jokes.csv has been successfully generated.")

all_jokes.csv has been successfully generated.


In [6]:
# Check if length of all_jokes_df is equal to the sum
# of lengths of the three dataframes

total_length = len(stupidstuff_df) + len(wocka_df) + len(reddit_jokes_df)
if len(all_jokes_df) == total_length:
    print("Length of all_jokes_df matches the sum of lengths of the three dataframes.")

# Display the columns for all_jokes.csv
print("\nall_jokes.csv columns:")
print(all_jokes_df.columns)

# Check if number of missing values in the 'body' column
# of all_jokes_df is equal to the sum of missing values
# in the 'body' columns of the three dataframes

missing_values = all_jokes_df['body'].isnull().sum()
missing_values_stupidstuff = stupidstuff_df['body'].isnull().sum()
missing_values_wocka = wocka_df['body'].isnull().sum()
missing_values_reddit_jokes = reddit_jokes_df['body'].isnull().sum()

if missing_values == missing_values_stupidstuff + missing_values_wocka + missing_values_reddit_jokes:
    print("Number of missing values in all_jokes_df matches the sum of missing values in the three dataframes.")

Length of all_jokes_df matches the sum of lengths of the three dataframes.

all_jokes.csv columns:
Index(['body'], dtype='object')
Number of missing values in all_jokes_df matches the sum of missing values in the three dataframes.


In [7]:
# Drop all the values with missing 'body' column
all_jokes_df.dropna(subset=['body'], inplace=True)

# Shuffle the rows of the dataframe
all_jokes_df = all_jokes_df.sample(frac=1).reset_index(drop=True)

Clean the dataset:

- Remove Special Characters
- Lower casing
- Remove Extra Spaces

In [8]:
# Combine all jokes into a single string
all_text = ' '.join(all_jokes_df['body'].astype(str))
# Find all unique non-alphanumeric characters
non_alphanumeric_chars = set(re.findall(r'[^a-zA-Z0-9]', all_text))
# Report the non-alphanumeric characters
print("Non-alphanumeric characters found:", non_alphanumeric_chars)

Non-alphanumeric characters found: {'\r', '\u2000', '\u2028', '\u3000', '\xa0', ' ', '\u2009', '\x85', '\u200a', '\t', '\n'}


In [9]:
all_jokes_df.head()

,body
0,What is the best joke that made you LOL
1,I was having sex with a female amputee in the ...
2,Being a bachelor is dangerous I pulled a groin...
3,Everythings racist these days You cant even sa...
4,What did Stevie Wonder say when he tried marih...


In [10]:
# Enable progress bar for the apply function
tqdm.pandas()

# make everything in body column lowercase and save the csv file
all_jokes_df['body'] = all_jokes_df['body'].progress_apply(lambda x: x.lower())
all_jokes_df.to_csv(os.path.join('all_jokes.csv'), index=False)

100%|██████████| 207761/207761 [00:00<00:00, 1715116.17it/s]


In [11]:
# Remove the characters '\u2000', '\u200a', '\u2028', '\xa0', '\t', '\x85', '\u3000', '\u2009' and '\n'
chars_to_remove = ['\u2000', '\u200a', '\u2028', '\xa0', '\t', '\x85', '\u3000', '\u2009', '\n', '\r']

tqdm.pandas()
# Remove the characters by replacing them with a space
all_jokes_df['body'] = all_jokes_df['body'].progress_apply(lambda x: ''.join([char if char not in chars_to_remove else ' ' for char in x]))

# Save the cleaned data to a new CSV file
all_jokes_df.to_csv(os.path.join('all_jokes.csv'), index=False)

100%|██████████| 207761/207761 [00:03<00:00, 63688.62it/s]


In [12]:
# Enable progress bar for pandas operations
tqdm.pandas(desc="Normalizing spaces in jokes")

# Normalize spaces in the body column using regex
# This will replace any number of consecutive whitespace characters with a single space
all_jokes_df['body'] = all_jokes_df['body'].progress_apply(
    lambda x: re.sub(r'\s+', ' ', str(x)).strip()
)

# Save the cleaned data
all_jokes_df.to_csv('jokes_normalized.csv', index=False)
print("Completed! Data saved to jokes_normalized.csv")

# Print a sample to verify
print("\nSample of normalized texts:")
print(all_jokes_df['body'].head())

Normalizing spaces in jokes: 100%|██████████| 207761/207761 [00:01<00:00, 150453.26it/s]


Completed! Data saved to jokes_normalized.csv

Sample of normalized texts:
0              what is the best joke that made you lol
1    i was having sex with a female amputee in the ...
2    being a bachelor is dangerous i pulled a groin...
3    everythings racist these days you cant even sa...
4    what did stevie wonder say when he tried marih...
Name: body, dtype: object


In [13]:
# Backup the original file
if os.path.exists('all_jokes.csv'):
    shutil.copy2('all_jokes.csv', 'all_jokes_backup.csv')
    print("Original file backed up as 'all_jokes_backup.csv'")

# Replace the original file with the normalized version
shutil.move('jokes_normalized.csv', 'all_jokes.csv')
print("Successfully replaced 'all_jokes.csv' with the normalized version")

Original file backed up as 'all_jokes_backup.csv'
Successfully replaced 'all_jokes.csv' with the normalized version


In [14]:
# Check a few samples
print(all_jokes_df['body'].head())

# count spaces in a row to verify
print(all_jokes_df['body'].iloc[0].count('  '))

0              what is the best joke that made you lol
1    i was having sex with a female amputee in the ...
2    being a bachelor is dangerous i pulled a groin...
3    everythings racist these days you cant even sa...
4    what did stevie wonder say when he tried marih...
Name: body, dtype: object
0


In [15]:
# Find all the unique characters in the 'body' column

all_text = ' '.join(all_jokes_df['body'].astype(str))
unique_chars = set(all_text)
print("Unique characters in the 'body' column:", unique_chars)
print(len(unique_chars))

Unique characters in the 'body' column: {'5', 'h', 'l', '2', 's', 't', 'r', 'g', 'f', 'v', '4', 'o', 'z', 'p', 'k', 'i', '8', '1', 'e', ' ', 'b', 'd', '7', 'y', 'c', 'a', 'm', 'w', 'j', 'x', '3', 'q', '9', '0', 'n', '6', 'u'}
37


In [16]:
# Make a list of the body column

random.seed(40)
jokes_list = all_jokes_df['body'].tolist()
print(len(jokes_list))

# Remove any empty strings and strings with length more than 128
jokes_list = [joke for joke in jokes_list if len(joke) <= 128]
print(len(jokes_list))

207761
134348


In [17]:
# Set a fixed seed to ensure reproducibility
random.seed(42)

# Shuffle the jokes list and get the first 5 jokes
random.shuffle(jokes_list)
jokes_list[:5]

['ive spent all day trying to convince people on reddit im french i give up',
 'time for a police based knock knock joke pew pew pew its the police open the door',
 'a duck walks into a bar animal control is promptly called and the duck is released in a nearby park',
 'my dentist offered to give me dentures for only a dollar it sounded like a good deal at the time but now i have buck teeth',
 'i kidnapped this girl last night and she yelled please i dont want to die a virgin if that isnt consent i dont know what is']

In [18]:
def count_unique_words(sentences):
    # Split each sentence into words and make lowercase
    all_words = [word.lower() for sentence in sentences for word in sentence.split()]
    # Convert to set to get unique words
    unique_words = set(all_words)
    return len(unique_words)

num_unique = count_unique_words(jokes_list)
print(f"Number of unique words: {num_unique}")

Number of unique words: 53735


In [19]:
# Initial split: 80% training, 20% for test + validation
train_jokes, temp_jokes = train_test_split(
    jokes_list,
    test_size=0.2,
    random_state=42
)

# Second split: split the remaining 20% into validation (10%) and test (10%)
val_jokes, test_jokes = train_test_split(
    temp_jokes,
    test_size=0.5,
    random_state=42
)

# Create HuggingFace datasets
train_dataset = Dataset.from_dict({'text': train_jokes})
val_dataset = Dataset.from_dict({'text': val_jokes})
test_dataset = Dataset.from_dict({'text': test_jokes})

# Print dataset sizes to verify splits
print(f"Training set size: {len(train_dataset)} ({len(train_dataset)/len(jokes_list)*100:.1f}%)")
print(f"Validation set size: {len(val_dataset)} ({len(val_dataset)/len(jokes_list)*100:.1f}%)")
print(f"Test set size: {len(test_dataset)} ({len(test_dataset)/len(jokes_list)*100:.1f}%)")

# Optional: save datasets
train_dataset.save_to_disk('train_dataset')
val_dataset.save_to_disk('val_dataset')
test_dataset.save_to_disk('test_dataset')

Training set size: 107478 (80.0%)
Validation set size: 13435 (10.0%)
Test set size: 13435 (10.0%)


Saving the dataset (1/1 shards): 100%|██████████| 13435/13435 [00:00<00:00, 4762951.08 examples/s]


In [20]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2") # Load GPT-2 tokenizer
tokenizer.pad_token = tokenizer.eos_token # Set the pad token to the end-of-sequence token

# Define tokenization function
def tokenize_function(examples): # Function to tokenize the text
    return tokenizer( # Tokenize the text
        examples['text'], # Extract the 'text' field
        truncation=True, # Truncate the text if it exceeds the maximum length
        padding=True, # Pad the text to the maximum length
        max_length=512,  # Adjust this value if needed
        return_tensors=None  # Return lists instead of tensors (required for batching)
    )

# Tokenize all datasets
tokenized_train = train_dataset.map( # Tokenize the training set
    tokenize_function, # Use the tokenization function
    batched=True, # Tokenize in batches
    desc="Tokenizing training set" # Description for the progress bar
)

tokenized_val = val_dataset.map( # Tokenize the validation set
    tokenize_function, # Use the tokenization function
    batched=True, # Tokenize in batches
    desc="Tokenizing validation set" # Description for the progress bar
)

tokenized_test = test_dataset.map( # Tokenize the test set
    tokenize_function, # Use the tokenization function
    batched=True, # Tokenize in batches
    desc="Tokenizing test set" # Description for the progress bar
)

# Create data collator for dynamic padding
data_collator = DataCollatorWithPadding( # Create a data collator
    tokenizer=tokenizer, # Set the tokenizer
    padding=True, # Enable padding
    return_tensors="pt" # Return PyTorch tensors
)

# Print sample output to verify
print("\nSample from tokenized training set:")
print("Input IDs shape:", len(tokenized_train[0]['input_ids']))
print("Attention mask shape:", len(tokenized_train[0]['attention_mask']))

# Save tokenized datasets
tokenized_train.save_to_disk('tokenized_train')
tokenized_val.save_to_disk('tokenized_val')
tokenized_test.save_to_disk('tokenized_test')

# Print dataset sizes
print("\nDataset sizes:")
print(f"Training set: {len(tokenized_train)} examples")
print(f"Validation set: {len(tokenized_val)} examples")
print(f"Test set: {len(tokenized_test)} examples")

Tokenizing test set: 100%|██████████| 13435/13435 [00:00<00:00, 95193.85 examples/s]



Sample from tokenized training set:
Input IDs shape: 33
Attention mask shape: 33


Saving the dataset (1/1 shards): 100%|██████████| 13435/13435 [00:00<00:00, 2691558.76 examples/s]


Dataset sizes:
Training set: 107478 examples
Validation set: 13435 examples
Test set: 13435 examples


In [21]:
# Randomly drop 60% of the data since the kernel is crashing without training being finished if the full dataset is used
tokenized_train = tokenized_train.filter(lambda example, idx: random.random() < 0.4, with_indices=True)
tokenized_val = tokenized_val.filter(lambda example, idx: random.random() < 0.4, with_indices=True)
tokenized_test = tokenized_test.filter(lambda example, idx: random.random() < 0.4, with_indices=True)

Filter: 100%|██████████| 13435/13435 [00:00<00:00, 77224.91 examples/s]


In [22]:
random.seed(40)
test_samples = random.sample(test_jokes, 5)

In [23]:
test_samples

['whats long and hard and full of semen a submarine',
 'did you hear about the knifewielding madman who attacked a circus camp the other day he went straight for the juggler',
 'would you like to make me wine join everyone stomping my grapes',
 'yo mamas so fat she fell in love and broke it',
 'i hate the word chicks can we politely call them ladies women please ladies nuts on your chin']

### Part 2

Create a decoder model and train it using the joke-dataset. Try out different hyperparameters for the model, like the number of decoder blocks, hidden dimension, sequence length etc. Choose the model which gives you the best results. Log the results for the different hyperparameters combinations used. Submit the logs in a pdf. (18 marks)


In [24]:
class CausalSelfAttention(nn.Module): # Define the CausalSelfAttention class
  def __init__(self, d_k,d_model, n_heads, max_len): # Constructor
    super().__init__() # Initialize the base class
    # Assume d_v =d_k
    self.d_k=d_k # Set the value of d_k
    self.n_heads = n_heads # Set the number of heads
    self.key = nn. Linear (d_model, d_k*n_heads) # Linear layer for key
    self.query = nn. Linear (d_model, d_k*n_heads) # Linear layer for query
    self.value = nn. Linear (d_model, d_k*n_heads) # Linear layer for value
    # final linear layer
    self.fc=nn.Linear (d_k*n_heads, d_model) # Linear layer for the final output
    # casual mask
    # make it so that diagonal is 0
    # this way we don't have to shift the inputs to make targets
    cm=torch.tril(torch.ones(int(max_len),int(max_len))) # Lower triangular matrix
    self.register_buffer("causal_mask",  cm.view(1,1, int(max_len), int(max_len))) # Register the causal mask as a buffer

  def forward(self, q, k, v, pad_mask=None): # Forward pass
    q=self.query(q) # N x T x (hd_k)
    k=self.key(k) # N x T x (hd_k)
    v=self.value(v) # N x T x (hd_k)
    N = q.shape[0] # Batch size
    T = q.shape[1] # Sequence length
    # change the shape to:
    # (N, T, h, d_k) --> N, h, T, d_k)
    # in order for matrix multiply to work properly
    q=q.view (N, T, self.n_heads, self.d_k).transpose(1,2) # Transpose q
    k=k.view (N, T, self.n_heads, self.d_k).transpose(1,2) # Transpose k
    v=v.view (N, T, self.n_heads, self.d_k).transpose(1,2) # Transpose v
    # Copute attention weights
    # (N, h, T, d_k)   x  (N, h, d_k, T )  --> (N, h, T, T)
    attn_scores = q@k.transpose(-2,-1)/math.sqrt(self.d_k) # Scaled dot product;  @ --> torch.matmul
    if pad_mask is not None: # Check if a padding mask is provided
      attn_scores = attn_scores.masked_fill(pad_mask[:,None,None,:] == 0, float('-inf')) # Mask the padding tokens
    attn_scores = attn_scores.masked_fill(self.causal_mask[:, :, :T, :T] == 0, float('-inf')) # Mask the future tokens
    attn_weights = F.softmax(attn_scores, dim =-1) # Compute the attention weights
    # Compute attention-weighted values
    # (N, h, T, T) x (N, h, T, d_k) --> (N, h, T, d_k)
    A = attn_weights @ v # Matrix multiplication
    # reshape it back before final linear layer
    A = A.transpose(1,2)  # (N, T, h, d_k) --> (N, T, h, d_k)
    A = A.contiguous(). view(N, T, self.d_k*self.n_heads) # (N, T, h*d_k) --> (N, T, hd_k)
    # projection
    return self.fc(A)

In [25]:
class TransformerBlock(nn.Module): # Define the TransformerBlock class
  def __init__(self, d_k, d_model, n_heads, max_len, dropout_prob = 0.1): # Constructor
    super().__init__() # Initialize the base class
    self.ln1 = nn.LayerNorm(d_model) # Layer normalization
    self.ln2 = nn.LayerNorm(d_model) # Layer normalization
    self.mha = CausalSelfAttention(d_k, d_model, n_heads, max_len) # Multi-head attention
    self.ann = nn.Sequential( # Feedforward neural network
        nn.Linear(d_model, d_model*4), # Linear layer
        nn.GELU(), # GELU activation
        nn.Linear(d_model*4, d_model), # Linear layer
        nn.Dropout(dropout_prob), # Dropout layer
    ) # End of the feedforward neural network
    self.dropout = nn.Dropout(p=dropout_prob) # Dropout layer

  def forward (self, x, pad_mask=None): # Forward pass
    x = self.ln1(x + self.mha(x, x, x, pad_mask)) # Multi-head attention
    x = self.ln2(x + self.ann(x)) # Feedforward neural network
    x = self.dropout(x) # Dropout
    return x

In [26]:
class PositionalEncoding(nn.Module): # Define the PositionalEncoding class
  def __init__(self, d_model, max_len =2048, dropout_prob=0.1): # Constructor
    super().__init__() # Initialize the base class
    self.dropout = nn.Dropout(p = dropout_prob) # Dropout layer
    position = torch.arange(max_len).unsqueeze(1) # Positional encoding
    i = torch.arange(0, d_model//2) # Index
    pe = torch.zeros(1, max_len, d_model) # Initialize the positional encoding
    pe[0, :, 0::2] = torch.sin(position / (10000)**(2*i/d_model)) # Compute the positional encoding
    pe[0, :, 1::2] = torch.cos(position / (10000)**(2*i/d_model)) # Compute the positional encoding
    self.register_buffer('pe', pe) # Register the positional encoding as a buffer

  def forward(self, x): # Forward pass
    # x.shape : N x T x D
    x = x + self.pe[:, :x.size(1), :] # Add positional encoding
    return self.dropout(x) # Return the output

In [27]:
class Decoder(nn.Module): # Define the Decoder class
  def __init__(self, vocab_size, max_len, d_k, d_model, n_heads, n_layers, dropout_prob): # Constructor
    super().__init__() # Initialize the base class
    self.embedding=nn.Embedding(vocab_size, d_model) # Embedding layer
    self.pos_encoding=PositionalEncoding(d_model, max_len, dropout_prob) # Positional encoding
    transformer_blocks=[TransformerBlock(d_k, d_model, n_heads, max_len, dropout_prob) for _ in range(n_layers) ] # Transformer blocks
    self.transformer_blocks = nn.Sequential(*transformer_blocks) # Sequential model
    self.ln = nn.LayerNorm(d_model) # Layer normalization
    self.fc = nn.Linear(d_model, vocab_size) # Linear layer

  def forward(self, x, pad_mask=None): # Forward pass
    x=self.embedding(x) # Embedding
    x=self.pos_encoding(x) # Positional encoding
    for block in self.transformer_blocks: # Iterate over the transformer blocks
      x = block(x, pad_mask) # Transformer block
    x = self.ln(x) # Layer normalization
    x = self.fc(x)  # many-to-many --> many-to-many
    return x

In [28]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# Show the device name
torch.cuda.get_device_name(0)

cuda:0


'NVIDIA GeForce GTX 1660 SUPER'

In [29]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [30]:
def create_transformer_decoder(d_k, d_model, n_heads, n_layers, dropout_prob): # Define the create_transformer_decoder function
    """
    Create and return a Transformer Decoder model based on the input hyperparameters.

    Args:
    vocab_size (int): The size of the vocabulary.
    max_len (int): Maximum length of the input sequence.
    d_k (int): Dimension of the keys/queries.
    d_model (int): Dimension of the model (hidden size).
    n_heads (int): Number of attention heads.
    n_layers (int): Number of Transformer layers.
    dropout_prob (float): Dropout probability.
    device (str): Device to run the model on (e.g., 'cpu' or 'cuda').

    Returns:
    nn.Module: The initialized Transformer Decoder model.
    """

    model = Decoder(
        vocab_size=tokenizer.vocab_size,
        max_len=tokenizer.model_max_length,
        d_k=d_k,
        d_model=d_model,
        n_heads=n_heads,
        n_layers=n_layers,
        dropout_prob=dropout_prob
    )

    # Move the model to the specified device (e.g., CPU or GPU)
    model.to(device)

    return model

In [31]:
def prepare_data_for_training(dataset, seq_length): # Define the prepare_data_for_training function
    """
    Prepare dataset for training by converting to appropriate format.
    Args:
        dataset: Dictionary containing 'input_ids' and 'attention_mask'
        seq_length: Target sequence length for padding/truncation
    Returns:
        TensorDataset with input_ids and attention_mask
    """
    input_ids = dataset['input_ids'] # Extract input IDs
    attention_mask = dataset['attention_mask'] # Extract attention mask

    # Initialize lists for padded sequences
    padded_inputs = [] # Initialize an empty list
    padded_masks = [] # Initialize an empty list

    # Process each sequence individually
    for seq, mask in zip(input_ids, attention_mask):
        # Convert to list if not already
        seq = list(seq)
        mask = list(mask)

        if len(seq) > seq_length: # Check if the sequence length exceeds the target length
            # Truncate
            padded_seq = seq[:seq_length] # Truncate the sequence
            padded_mask = mask[:seq_length] # Truncate the mask
        else:
            # Pad
            padding_length = seq_length - len(seq) # Calculate the padding length
            padded_seq = seq + [0] * padding_length  # Pad with zeros
            padded_mask = mask + [0] * padding_length # Pad with zeros

        padded_inputs.append(padded_seq) # Append the padded sequence
        padded_masks.append(padded_mask) # Append the padded mask

    # Convert to numpy arrays after padding
    padded_inputs = np.array(padded_inputs, dtype=np.int64) # Convert to a NumPy array
    padded_masks = np.array(padded_masks, dtype=np.int64) # Convert to a NumPy array

    return torch.utils.data.TensorDataset( # Return a TensorDataset
        torch.LongTensor(padded_inputs), # Convert to a PyTorch tensor
        torch.LongTensor(padded_masks) # Convert to a PyTorch tensor
    )

def calculate_l2_loss(model, lambda_l2=0.01): # Define the calculate_l2_loss function
    """
    Calculate L2 regularization loss for model parameters.
    """
    l2_loss = 0
    for param in model.parameters():
        l2_loss += torch.norm(param, p=2) # Calculate the L2 norm of the parameter
    return lambda_l2 * l2_loss

In [32]:
def train_transformer(
    d_k: int, # Dimension of key/query vectors
    d_model: int, # Model dimension
    n_heads: int, # Number of attention heads
    n_layers: int, # Number of transformer layers
    dropout_prob: float, # Dropout probability
    learning_rate: float, # Learning rate for optimizer
    num_epochs: int, # Number of training epochs
    batch_size: int, # Batch size for training
    seq_length: int, # Maximum sequence length for padding/truncation
    save_path: str = 'best_model.pt', # Path to save best model
    early_stopping_patience: int = 3, # Number of epochs to wait before early stopping
    weight_decay: float = 0.01, # L2 regularization factor for optimizer
    lambda_l2: float = 0.01 # L2 regularization factor for loss function
) -> Dict[str, Any]:
    """
    Train a transformer model with added L2 regularization and improved initialization.

    Args:
        d_k: Dimension of key/query/value vectors
        d_model: Model dimension
        n_heads: Number of attention heads
        n_layers: Number of transformer layers
        dropout_prob: Dropout probability
        learning_rate: Learning rate for optimizer
        num_epochs: Number of training epochs
        batch_size: Batch size for training
        seq_length: Maximum sequence length for padding/truncation
        save_path: Path to save best model
        early_stopping_patience: Number of epochs to wait before early stopping
        weight_decay: L2 regularization factor for optimizer
        lambda_l2: L2 regularization factor for loss function

    Returns:
        dict: Training history containing losses and metrics
    """
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Check if GPU is available
    print(f"Using device: {device}") # Print device

    # Create model
    model = create_transformer_decoder( # Create transformer model
        d_k=d_k, # Dimension of key/query vectors
        d_model=d_model, # Model dimension
        n_heads=n_heads, # Number of attention heads
        n_layers=n_layers, # Number of transformer layers
        dropout_prob=dropout_prob # Dropout probability
    ).to(device) # Move model to device

    # Initialize weights using Xavier/Glorot initialization
    def init_weights(m): # Initialize weights
        if isinstance(m, nn.Linear): # If linear layer
            torch.nn.init.xavier_uniform_(m.weight) # Use Xavier initialization
            if m.bias is not None: # If bias is not None
                torch.nn.init.zeros_(m.bias) # Initialize bias to zeros
    model.apply(init_weights) # Apply weight initialization

    # Log model parameters
    total_params = sum(p.numel() for p in model.parameters()) # Total parameters
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad) # Trainable parameters
    print(f"Total parameters: {total_params:,}") # Print total parameters
    print(f"Trainable parameters: {trainable_params:,}") # Print trainable parameters

    # Prepare datasets with specified sequence length
    train_data = prepare_data_for_training(tokenized_train, seq_length) # Prepare training data
    val_data = prepare_data_for_training(tokenized_val, seq_length) # Prepare validation data
    test_data = prepare_data_for_training(tokenized_test, seq_length) # Prepare test data

    # Create data loaders
    train_loader = torch.utils.data.DataLoader( # Training data loader
        train_data, # Training data
        batch_size=batch_size, # Batch size
        shuffle=True, # Shuffle data
        pin_memory=True, # Pin memory
        num_workers=2 # Number of workers
    )

    val_loader = torch.utils.data.DataLoader( # Validation data loader
        val_data, # Validation data
        batch_size=batch_size, # Batch size
        shuffle=False, # Do not Shuffle data
        pin_memory=True, # Pin memory
        num_workers=2 # Number of workers
    )

    test_loader = torch.utils.data.DataLoader( # Test data loader
        test_data, # Test data
        batch_size=batch_size, # Batch size
        shuffle=False, # Do not Shuffle data
        pin_memory=True, # Pin memory
        num_workers=2 # Number of workers
    )

    # Initialize optimizer with weight decay
    optimizer = torch.optim.AdamW( # AdamW optimizer
        model.parameters(), # Model parameters
        lr=learning_rate, # Learning rate
        weight_decay=weight_decay, # Weight decay
        betas=(0.9, 0.999), # Betas
        eps=1e-8 # Epsilon
    )

    criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id) # Cross entropy loss
    scheduler = ReduceLROnPlateau( # Reduce LR on plateau
        optimizer, # Optimizer
        mode='min', # Mode
        factor=0.5, # Factor
        patience=2, # Patience
        verbose=True, # Verbose
        min_lr=1e-7 # Min LR
    )

    # Training history
    history = {
        'train_loss': [], # Training loss
        'train_reg_loss': [], # Training regularization loss
        'val_loss': [], # Validation loss
        'val_perplexity': [], # Validation perplexity
        'learning_rates': [], # Learning rates
        'test_loss': None, # Test loss
        'test_perplexity': None # Test perplexity
    }

    # Early stopping variables
    best_val_loss = float('inf') # Best validation loss
    patience_counter = 0 # Patience counter
    best_epoch = 0 # Best epoch

    # Gradient scaler for mixed precision training
    scaler = torch.cuda.amp.GradScaler() if device.type == 'cuda' else None # Gradient scaler

    # Training loop
    for epoch in range(num_epochs): # Epoch loop
        model.train() # Set model to training mode
        total_train_loss = 0 # Total training loss
        total_reg_loss = 0 # Total regularization loss
        train_steps = 0 # Training steps

        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}') # Progress bar
        for batch_idx, (input_ids, attention_mask) in enumerate(progress_bar): # Batch loop
            try:
                # Move batch to device
                input_ids = input_ids.to(device) # Input IDs
                attention_mask = attention_mask.to(device) # Attention mask

                # Mixed precision training context
                with torch.cuda.amp.autocast() if device.type == 'cuda' else nullcontext():
                    # Forward pass
                    outputs = model(input_ids, attention_mask) # Model outputs

                    # Shift predictions and targets
                    shift_logits = outputs[..., :-1, :].contiguous() # Shift logits
                    shift_labels = input_ids[..., 1:].contiguous() # Shift labels

                    # Calculate cross entropy loss
                    ce_loss = criterion( # Cross entropy loss
                        shift_logits.view(-1, tokenizer.vocab_size), # Flattened logits
                        shift_labels.view(-1) # Flattened labels
                    )

                    # Add L2 regularization
                    l2_loss = calculate_l2_loss(model, lambda_l2) # L2 regularization loss
                    loss = ce_loss + l2_loss # Total loss

                # Backward pass with gradient scaling
                optimizer.zero_grad(set_to_none=True) # Zero gradients
                if scaler is not None: # Use gradient scaler
                    scaler.scale(loss).backward() # Backward pass
                    scaler.unscale_(optimizer) # Unscaled gradients
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Clip gradients
                    scaler.step(optimizer) # Step optimizer
                    scaler.update() # Update scaler
                else: # No gradient scaler
                    loss.backward() # Backward pass
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Clip gradients
                    optimizer.step() # Step optimizer

                # Update metrics
                if not torch.isnan(loss): # Skip NaN losses
                    total_train_loss += ce_loss.item() # Accumulate CE loss
                    total_reg_loss += l2_loss.item() # Accumulate L2 loss
                    train_steps += 1 # Increment train steps

                    # Update progress bar
                    progress_bar.set_postfix({ # Update progress bar
                        'ce_loss': ce_loss.item(), # CE loss
                        'l2_loss': l2_loss.item(), # L2 loss
                        'lr': optimizer.param_groups[0]['lr'] # Learning rate
                    })
                else: # NaN loss
                    print(f'Warning: NaN loss at epoch {epoch + 1}, batch {batch_idx}') # Print warning

            except RuntimeError as e: # Catch OOM errors
                print(f"Error in training batch {batch_idx}: {str(e)}") # Print error message
                continue # Skip the current batch

        # Calculate average losses
        avg_train_loss = total_train_loss / train_steps # Average training loss
        avg_reg_loss = total_reg_loss / train_steps # Average regularization loss
        history['train_loss'].append(avg_train_loss) # Log training loss
        history['train_reg_loss'].append(avg_reg_loss) # Log regularization loss
        history['learning_rates'].append(optimizer.param_groups[0]['lr']) # Log learning rate

        # Validation phase
        model.eval() # Set model to evaluation mode
        total_val_loss = 0 # Total validation loss
        val_steps = 0 # Validation steps

        with torch.no_grad(): # Disable gradients for validation
            for input_ids, attention_mask in tqdm(val_loader, desc='Validation'): # Validation loop
                try: # Try block
                    input_ids = input_ids.to(device) # Move to device
                    attention_mask = attention_mask.to(device) # Move to device
                    outputs = model(input_ids, attention_mask) # Forward pass
                    shift_logits = outputs[..., :-1, :].contiguous() # Shift logits
                    shift_labels = input_ids[..., 1:].contiguous() # Shift labels

                    loss = criterion( # Calculate validation loss
                        shift_logits.view(-1, tokenizer.vocab_size), # Flatten logits
                        shift_labels.view(-1) # Flatten labels
                    )

                    if not torch.isnan(loss): # Skip NaN losses
                        total_val_loss += loss.item() # Accumulate validation loss
                        val_steps += 1 # Increment validation steps

                except RuntimeError as e: # Catch OOM errors
                    print(f"Error in validation batch: {str(e)}") # Print error message
                    continue # Skip the current batch

        # Calculate validation metrics
        avg_val_loss = total_val_loss / val_steps # Average validation loss
        val_perplexity = torch.exp(torch.tensor(avg_val_loss)) # Validation perplexity

        history['val_loss'].append(avg_val_loss) # Log validation loss
        history['val_perplexity'].append(val_perplexity.item()) # Log validation perplexity

        # Update learning rate scheduler
        scheduler.step(avg_val_loss) # Step scheduler

        # Print epoch results
        print(f'\nEpoch {epoch + 1} Results:') # Print epoch results
        print(f'Training CE loss: {avg_train_loss:.4f}') # Print training loss
        print(f'Training L2 loss: {avg_reg_loss:.4f}') # Print regularization loss
        print(f'Validation loss: {avg_val_loss:.4f}') # Print validation loss
        print(f'Validation perplexity: {val_perplexity:.4f}') # Print validation perplexity
        print(f'Learning rate: {optimizer.param_groups[0]["lr"]:.2e}') # Print learning rate

        # Model checkpointing and early stopping
        if avg_val_loss < best_val_loss: # Save best model
            best_val_loss = avg_val_loss # Update best validation loss
            best_epoch = epoch + 1 # Update best epoch
            torch.save({ # Save checkpoint
                'epoch': epoch, # Epoch
                'model_state_dict': model.state_dict(), # Model state
                'optimizer_state_dict': optimizer.state_dict(), # Optimizer state
                'scheduler_state_dict': scheduler.state_dict(), # Scheduler state
                'val_loss': best_val_loss, # Validation loss
            }, save_path) # Save path
            patience_counter = 0 # Reset patience counter
            print(f'Saved new best model to {save_path}') # Print save message
        else: # No improvement
            patience_counter += 1 # Increment patience counter
            if patience_counter >= early_stopping_patience: # Early stopping
                print(f'\nEarly stopping triggered after {epoch + 1} epochs') # Print early stopping message
                print(f'Best validation loss: {best_val_loss:.4f} at epoch {best_epoch}') # Print best validation loss
                break # Break the training loop

    # Load best model for testing
    print("\nLoading best model for final evaluation...") # Load best model
    checkpoint = torch.load(save_path) # Load checkpoint
    model.load_state_dict(checkpoint['model_state_dict']) # Load model state
    model.eval() # Set model to evaluation mode

    # Test set evaluation
    print("Evaluating on test set...") # Test phase
    total_test_loss = 0 # Total test loss
    test_steps = 0 # Number of test steps

    with torch.no_grad(): # Disable gradients for testing
        for input_ids, attention_mask in tqdm(test_loader, desc='Testing'): # Forward pass
            try: # Forward pass
                input_ids = input_ids.to(device) # Move to device
                attention_mask = attention_mask.to(device) # Forward pass

                outputs = model(input_ids, attention_mask) # Forward pass

                shift_logits = outputs[..., :-1, :].contiguous() # Ignore last token
                shift_labels = input_ids[..., 1:].contiguous() # Ignore first token

                loss = criterion( # Calculate test loss
                    shift_logits.view(-1, tokenizer.vocab_size), 
                    shift_labels.view(-1)# Flatten the target sequence
                )

                if not torch.isnan(loss): # Skip NaN losses
                    total_test_loss += loss.item() # Accumulate test loss
                    test_steps += 1 # Increment test steps

            except RuntimeError as e: # Catch OOM errors
                print(f"Error in test batch: {str(e)}") # Print error message
                continue # Skip the current batch

    # Calculate and log final metrics
    avg_test_loss = total_test_loss / test_steps # Calculate average test loss
    test_perplexity = torch.exp(torch.tensor(avg_test_loss)) # Calculate test perplexity

    history['test_loss'] = avg_test_loss # Log test loss
    history['test_perplexity'] = test_perplexity.item() # Log test perplexity

    print('\nFinal Test Results:') # Print final test results
    print(f'Test loss: {avg_test_loss:.4f}') # Print test loss
    print(f'Test perplexity: {test_perplexity:.4f}') # Print test perplexity
    print(f'Best validation loss: {best_val_loss:.4f} at epoch {best_epoch}') # Print best validation loss

    return history # Return training history

In [33]:
os.environ["TOKENIZERS_PARALLELISM"] = "false" # Disable tokenizers parallelism

For the experiments, use less data points. After the hyperparameter tuning has been done, the model with best hyperparameters will be trained on the whole dataset.

In [34]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True" # Set CUDA memory management to expandable segments

In [35]:
def experiment_tracker( # Experiment tracker
    hidden_dims: List[int], # Hidden dimensions
    hidden_layers: List[int], # Hidden layers
    n_heads_list: List[int], # Number of heads
    seq_length: List[int], # Sequence length
    base_save_path: str = 'experiments' # Base save path
) -> str: # Return type
    """
    Run experiments with different hyperparameter combinations and track results.

    Args:
        hidden_dims: List of hidden dimensions to try
        hidden_layers: List of number of layers to try
        n_heads_list: List of number of heads to try
        seq_length: List of sequence lengths to try for padding/truncation
        base_save_path: Base directory to save experiments

    Returns:
        str: Path to the best overall model
    """
    # Check and print CUDA availability
    cuda_available = torch.cuda.is_available() # Check if CUDA is available
    print(f"CUDA Available: {cuda_available}") # Print CUDA availability

    # Create experiment directory with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S') # Timestamp
    exp_dir = os.path.join(base_save_path, f'experiment_{timestamp}') # Experiment directory
    os.makedirs(exp_dir, exist_ok=True) # Create experiment directory

    # Initialize PDF for logging
    pdf = FPDF() # Initialize PDF
    pdf.add_page() # Add page
    pdf.set_font("Arial", size=12) # Set font

    # Store all results
    all_results = [] # All results
    skipped_experiments = [] # Skipped experiments

    # Generate all combinations of hyperparameters
    combinations = list(itertools.product(hidden_layers, hidden_dims, n_heads_list, seq_length)) # Generate combinations
    total_experiments = len(combinations) # Total experiments

    # Log experiment setup
    setup_info = f""" # Log experiment setup
    Experiment Started at: {timestamp} # Experiment timestamp
    Number of combinations: {total_experiments} # Total combinations
    Hidden layers tested: {hidden_layers} # Hidden layers
    Hidden dimensions tested: {hidden_dims} # Hidden dimensions
    Number of heads tested: {n_heads_list} # Number of heads
    Sequence lengths tested: {seq_length} # Sequence lengths
    CUDA Available: {cuda_available} # CUDA availability
    """
    pdf.multi_cell(0, 10, setup_info) # Multi-cell print

    best_test_perplexity = float('inf') # Best test perplexity
    best_model_path = None # Best model path
    best_model_hyperparams = None # Best model hyperparameters

    print("\nStarting experiments...") # Print start message
    print(f"Total combinations to test: {total_experiments}") # Print total combinations
    print("Parameters being tested:") # Print parameters
    print(f"- Hidden layers: {hidden_layers}") # Print hidden layers
    print(f"- Hidden dimensions: {hidden_dims}") # Print hidden dimensions
    print(f"- Number of heads: {n_heads_list}") # Print number of heads
    print(f"- Sequence lengths: {seq_length}\n") # Print sequence lengths

    # Run experiments for each combination
    for experiment_num, (n_layers, d_model, n_heads, seq_len) in enumerate(combinations, 1): # Experiment loop
        print(f"\nExperiment {experiment_num}/{total_experiments}") # Print experiment number
        print("Current parameters being tested:") # Print current parameters
        print(f"- Number of layers: {n_layers}") # Print number of layers
        print(f"- Hidden dimension: {d_model}") # Print hidden dimension
        print(f"- Number of heads: {n_heads}") # Print number of heads
        print(f"- Sequence length: {seq_len}") # Print sequence length

        # Clear CUDA cache before each experiment
        if cuda_available: # If CUDA is available
            torch.cuda.empty_cache() # Clear CUDA cache

        # Check if sufficient GPU memory is available
        try:
            # Simulate model memory allocation to check available memory
            model_memory_estimate = (
                d_model * 4 *  # Input tensor
                (n_layers * 3 +  # Transformer layers
                 n_layers * 2)  # Other components
            )
            available_memory = torch.cuda.get_device_properties(0).total_memory # Total memory

            if model_memory_estimate > available_memory * 0.98:  # 9% of total memory
                skipped_info = ( # Skipped info
                    f"Skipping Experiment {experiment_num}/{total_experiments}: Insufficient GPU memory\n" # Skipped message
                    f"Layers: {n_layers}, Dim: {d_model}, Heads: {n_heads}, Seq Length: {seq_len}\n" # Parameters
                    f"Estimated Memory: {model_memory_estimate / (1024**2):.2f} MB\n" # Estimated memory
                    f"Available Memory: {available_memory / (1024**2):.2f} MB" # Available memory
                )
                skipped_experiments.append(skipped_info) # Append skipped info
                print(skipped_info) # Print skipped info
                continue # Skip the current experiment

        except RuntimeError as e: # Catch memory allocation errors
            skipped_info = ( # Skipped info
                f"Skipping Experiment {experiment_num}/{total_experiments} due to memory allocation error\n" # Skipped message
                f"Layers: {n_layers}, Dim: {d_model}, Heads: {n_heads}, Seq Length: {seq_len}\n" # Parameters
                f"Error: {str(e)}" # Error message
            )
            skipped_experiments.append(skipped_info) # Append skipped info
            print(skipped_info) # Print skipped info
            continue # Skip the current experiment

        # Create model-specific save path
        model_save_path = os.path.join( # Model save path
            exp_dir, # Experiment directory
            f'model_l{n_layers}_d{d_model}_h{n_heads}_s{seq_len}.pt' # Model path
        )

        # Capture stdout to redirect prints to PDF
        stdout_capture = io.StringIO() # Stdout capture
        with redirect_stdout(stdout_capture): # Redirect stdout
            print(f"\nExperiment {experiment_num}/{total_experiments}") # Print experiment number
            print(f"Parameters: layers={n_layers}, d_model={d_model}, n_heads={n_heads}, seq_length={seq_len}") # Print parameters

            # Call training function
            d_k = d_model // n_heads # Calculate key/query dimension
            try: # Try training
                history = train_transformer( # Train transformer
                    d_k=d_k, # Key/Query dimension
                    d_model=d_model, # Model dimension
                    n_heads=n_heads, # Number of heads
                    n_layers=n_layers, # Number of layers
                    dropout_prob=0.1, # Dropout probability
                    learning_rate=1e-5, # Learning rate
                    num_epochs=8, # Number of epochs
                    batch_size=8, # Batch size
                    seq_length=seq_len, # Sequence length
                    save_path=model_save_path, # Save path
                    weight_decay=0.01, # Weight decay
                    lambda_l2=0.01 # L2 regularization factor
                )
            except Exception as e: # Catch errors
                skipped_info = ( # Skipped info
                    f"Experiment {experiment_num}/{total_experiments} failed during training\n" # Skipped message
                    f"Layers: {n_layers}, Dim: {d_model}, Heads: {n_heads}, Seq Length: {seq_len}\n" # Parameters
                    f"Error: {str(e)}" # Error message
                )
                skipped_experiments.append(skipped_info) # Append skipped info
                print(skipped_info) # Print skipped info
                continue # Skip the current experiment

        # Get captured output
        training_log = stdout_capture.getvalue()

        # Add experiment results to PDF
        pdf.add_page() # Add page
        pdf.multi_cell(0, 10, f"\nExperiment {experiment_num}/{total_experiments} Results:") # Print experiment results
        pdf.multi_cell(0, 10, f"Parameters: layers={n_layers}, d_model={d_model}, n_heads={n_heads}, seq_length={seq_len}") # Print parameters
        pdf.multi_cell(0, 10, training_log) # Print training log

        # Track best model
        test_perplexity = history['test_perplexity'] # Test perplexity
        result = { # Result
            'n_layers': n_layers, # Number of layers
            'd_model': d_model, # Model dimension
            'n_heads': n_heads, # Number of heads
            'seq_length': seq_len, # Sequence length
            'test_perplexity': test_perplexity, # Test perplexity
            'model_path': model_save_path, # Model path
            'd_k': d_k # Key/Query dimension
        }
        all_results.append(result) # Append result

        if test_perplexity < best_test_perplexity: # Update best model
            best_test_perplexity = test_perplexity # Update best test perplexity
            best_model_path = model_save_path # Best model path
            best_model_hyperparams = result # Best model hyperparameters

    # Save the best model as best_best_model.pt
    if best_model_path: # If best model path
        final_best_path = os.path.join(exp_dir, 'best_best_model.pt') # Final best model path
        torch.save( # Save best model
            torch.load(best_model_path), # Load best model
            final_best_path # Final best model path
        )
    else: # No successful experiments
        final_best_path = None # No best model path
        print("No successful experiments completed.") # Print no successful experiments

    # Print best model hyperparameters
    if best_model_hyperparams: # If best model hyperparameters
        print("\nBest Model Hyperparameters:") # Print best model hyperparameters
        print(f"Layers: {best_model_hyperparams['n_layers']}") # Print number of layers
        print(f"Hidden Dimension: {best_model_hyperparams['d_model']}") # Print hidden dimension
        print(f"Number of Heads: {best_model_hyperparams['n_heads']}") # Print number of heads
        print(f"Sequence Length: {best_model_hyperparams['seq_length']}") # Print sequence length
        print(f"Key/Query Dimension (d_k): {best_model_hyperparams['d_k']}") # Print key/query dimension
        print(f"Test Perplexity: {best_model_hyperparams['test_perplexity']:.4f}") # Print test perplexity

    # Add summary to PDF
    pdf.add_page() # Add page
    pdf.multi_cell(0, 10, "\nExperiment Summary:") # Print experiment summary
    for result in sorted(all_results, key=lambda x: x['test_perplexity']): # Sort results
        summary = f""" # Summary
        Layers: {result['n_layers']} # Number of layers
        Hidden Dimension: {result['d_model']} # Hidden dimension
        Number of Heads: {result['n_heads']} # Number of heads
        Sequence Length: {result['seq_length']} # Sequence length
        Key/Query Dimension (d_k): {result['d_k']} # Key/Query dimension
        Test Perplexity: {result['test_perplexity']:.4f} # Test perplexity
        Model Path: {result['model_path']} # Model path
        """
        pdf.multi_cell(0, 10, summary) # Print summary

    # Add skipped experiments to PDF
    if skipped_experiments: # If skipped experiments
        pdf.add_page() # Add page
        pdf.multi_cell(0, 10, "\nSkipped Experiments:") # Print skipped experiments
        for skipped_exp in skipped_experiments: # Print skipped experiments
            pdf.multi_cell(0, 10, skipped_exp) # Print skipped experiments

    # Save final results
    results_path = os.path.join(exp_dir, 'experiment_results.json') # Results path
    with open(results_path, 'w') as f: # Save results
        json.dump({ # Save results
            'completed_experiments': all_results, # Completed experiments
            'skipped_experiments': skipped_experiments # Skipped experiments
        }, f, indent=4) # Save results

    # Save PDF
    pdf_path = os.path.join(exp_dir, 'experiment_log.pdf') # PDF path
    pdf.output(pdf_path) # Save PDF

    print(f"\nExperiment completed!") # Print completion message
    if final_best_path: # If best model path
        print(f"Best model saved to: {final_best_path}") # Print best model path
    print(f"Full logs saved to: {pdf_path}") # Print PDF path
    print(f"Results saved to: {results_path}") # Print results path

    return final_best_path # Return best model path

In [36]:
hidden_dims = [16, 32, 64] # Hidden dimensions
hidden_layers = [2, 4] # Hidden layers
n_heads_list = [2, 4] # Number of heads
seq_length = [256, 512] # Sequence length

best_model_path = experiment_tracker( # Experiment tracker
    hidden_dims=hidden_dims, # Hidden dimensions
    hidden_layers=hidden_layers, # Hidden layers
    n_heads_list=n_heads_list, # Number of heads
    seq_length=seq_length) # Sequence length

CUDA Available: True

Starting experiments...
Total combinations to test: 24
Parameters being tested:
- Hidden layers: [2, 4]
- Hidden dimensions: [16, 32, 64]
- Number of heads: [2, 4]
- Sequence lengths: [256, 512]


Experiment 1/24
Current parameters being tested:
- Number of layers: 2
- Hidden dimension: 16
- Number of heads: 2
- Sequence length: 256


Testing: 100%|██████████| 670/670 [00:08<00:00, 79.09it/s]



Experiment 2/24
Current parameters being tested:
- Number of layers: 2
- Hidden dimension: 16
- Number of heads: 2
- Sequence length: 512


Testing: 100%|██████████| 670/670 [00:17<00:00, 39.26it/s]



Experiment 3/24
Current parameters being tested:
- Number of layers: 2
- Hidden dimension: 16
- Number of heads: 4
- Sequence length: 256


...


Experiment 23/24
Current parameters being tested:
- Number of layers: 4
- Hidden dimension: 64
- Number of heads: 4
- Sequence length: 256


Testing: 100%|██████████| 670/670 [00:11<00:00, 59.68it/s]



Experiment 24/24
Current parameters being tested:
- Number of layers: 4
- Hidden dimension: 64
- Number of heads: 4
- Sequence length: 512


Testing: 100%|██████████| 670/670 [00:24<00:00, 27.91it/s]



Best Model Hyperparameters:
Layers: 2
Hidden Dimension: 64
Number of Heads: 4
Sequence Length: 512
Key/Query Dimension (d_k): 16
Test Perplexity: 1.3115

Experiment completed!
Best model saved to: experiments/experiment_20241112_163001/best_best_model.pt
Full logs saved to: experiments/experiment_20241112_163001/experiment_log.pdf
Results saved to: experiments/experiment_20241112_163001/experiment_results.json


In [37]:
# clear the torch cache

torch.cuda.empty_cache() # Clear CUDA cache
gc.collect() # Garbage collect
torch.cuda.empty_cache() # Clear CUDA cache

In [38]:
best_model_path

'experiments/experiment_20241112_163001/best_best_model.pt'

### Best Model Hyperparameters:

Layers: 2

Hidden Dimension: 64

Number of Heads: 4

Sequence Length: 512

Key/Query Dimension (d_k): 16

In [39]:
best_model = Decoder(
    vocab_size=tokenizer.vocab_size,
    max_len=tokenizer.model_max_length,
    d_k=16, # Key/Query dimension
    d_model=64, # Hidden dimension
    n_heads=4, # Number of attention heads
    n_layers=2, # Number of transformer layers
    dropout_prob=0.1
)

In [40]:
# Check if it matches the best model path

checkpoint = torch.load(best_model_path)
best_model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

### Part 3

Choose any 5 samples from the test dataset and generate the model outputs. (2 marks)

In [41]:
def custom_generate(
    model,
    input_ids,
    attention_mask,
    max_length=1024,
    num_return_sequences=3,
    temperature=1.0,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2
):
    """
    Enhanced text generation function with additional control mechanisms

    Args:
        model: Pre-trained language model
        input_ids: Input token IDs
        attention_mask: Attention mask for input tokens
        max_length: Maximum generated sequence length
        num_return_sequences: Number of alternative sequences to generate
        temperature: Controls randomness of prediction
        top_k: Number of highest probability tokens to keep
        top_p: Nucleus sampling threshold
        repetition_penalty: Penalty for repeating tokens
    """
    model.eval()
    device = input_ids.device
    generated_sequences = []
    eos_token_id = 50256  # GPT-2 end-of-sequence token

    with torch.no_grad():
        for batch_idx in range(input_ids.size(0)):
            single_input_ids = input_ids[batch_idx:batch_idx+1]
            single_attention_mask = attention_mask[batch_idx:batch_idx+1]

            batch_sequences = []
            for _ in range(num_return_sequences):
                current_ids = single_input_ids.clone()
                current_mask = single_attention_mask.clone()

                generation_attempts = 0
                max_generation_attempts = 10

                while (current_ids.shape[1] < max_length and
                       generation_attempts < max_generation_attempts):
                    generation_attempts += 1

                    try:
                        # Model prediction
                        outputs = model(current_ids, current_mask)
                        logits = outputs[:, -1, :]

                        # Temperature scaling
                        logits = logits / temperature

                        # Repetition penalty
                        if generation_attempts > 1:
                            for prev_token in current_ids[0]:
                                logits[:, prev_token] /= repetition_penalty

                        # Top-k and Top-p filtering
                        top_k_values, top_k_indices = torch.topk(logits, min(top_k, logits.size(-1)), dim=-1)

                        # Nucleus sampling (Top-p)
                        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                        cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)

                        # Remove tokens with cumulative probability above top_p
                        sorted_indices_to_remove = cumulative_probs > top_p
                        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
                        sorted_indices_to_remove[..., 0] = 0

                        indices_to_remove = sorted_indices[sorted_indices_to_remove]
                        logits[:, indices_to_remove] = float('-inf')

                        # Sample next token
                        probabilities = torch.softmax(logits, dim=-1)
                        probabilities = torch.nan_to_num(probabilities, nan=0.0)
                        probabilities = torch.clamp(probabilities, min=0, max=1)
                        probabilities /= probabilities.sum(dim=-1, keepdim=True)

                        next_token = torch.multinomial(probabilities, num_samples=1)

                        # Append token and update masks
                        current_ids = torch.cat([current_ids, next_token], dim=-1)
                        current_mask = torch.cat([
                            current_mask,
                            torch.ones_like(next_token, dtype=torch.long)
                        ], dim=-1)

                        # Stop generation conditions
                        if (next_token == eos_token_id).any() or current_ids.shape[1] >= max_length:
                            break

                    except Exception as e:
                        print(f"Generation error: {e}")
                        break

                batch_sequences.append(current_ids)

            generated_sequences.append(torch.cat(batch_sequences, dim=0))

    return torch.cat(generated_sequences, dim=0)

In [42]:
best_hyperparameter_model = create_transformer_decoder(
    d_k=1,
    d_model=4,
    n_heads=4,
    n_layers=4,
    dropout_prob=0.1
)

In [43]:
best_model

Decoder(
  (embedding): Embedding(50257, 64)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (mha): CausalSelfAttention(
        (key): Linear(in_features=64, out_features=64, bias=True)
        (query): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (fc): Linear(in_features=64, out_features=64, bias=True)
      )
      (ann): Sequential(
        (0): Linear(in_features=64, out_features=256, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=256, out_features=64, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05,

In [ ]:
best_model.to(device)

random.seed(40)
test_samples = random.sample(test_jokes, 5)

# Split each joke into two halves
half_test_samples = [joke[:len(joke)//2] for joke in test_samples]

test_samples_tokenized = tokenizer(
    half_test_samples,  # Use only the first half of each joke
    truncation=True,
    padding=True,
    max_length=512,  # seq_lengths
    return_tensors='pt'
)

input_ids = test_samples_tokenized['input_ids'].to(device)
attention_mask = test_samples_tokenized['attention_mask'].to(device)

try:
    outputs = custom_generate(
        best_model,
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=1024,  # seq_lengths
        num_return_sequences=3,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2
    )

    generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    for i, (joke, continuation) in enumerate(zip(test_samples, generated_texts)):
        print(f"Joke {i+1}:")   
        print(joke)
        print(" ".join(continuation))
        print()

except Exception as e:
    print(f"Generation error: {e}")

Joke 1:
whats long and hard and full of semen a submarine
w h a t s   l o n g   a n d   h a r d   a n d   ! ! ! ! ! ! a r d o n   M e l l l e i g h   a

Joke 2:
did you hear about the knifewielding madman who attacked a circus camp the other day he went straight for the juggler
w h a t s   l o n g   a n d   h a r d   a n d   ! ! ! ! ! P R O   t o   a   t h e   V a r i o u s

Joke 3:
would you like to make me wine join everyone stomping my grapes
w h a t s   l o n g   a n d   h a r d   a n d   ! ! ! ! ! !   M a r i o   t h e   a   A g e

Joke 4:
yo mamas so fat she fell in love and broke it
d i d   y o u   h e a r   a b o u t   t h e   k n i f e w i e l d i n g   m a d m a n   w h o   a t t a c k e d   a   a   t o ! ! ! ! ! C a   t h e   d o

Joke 5:
i hate the word chicks can we politely call them ladies women please ladies nuts on your chin
d i d   y o u   h e a r   a b o u t   t h e   k n i f e w i e l d i n g   m a d m a n   w h o   a t t a c k e d   a   i   a a s   o f   a n d F o 

In [69]:
# Clean up the GPU memory
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()

## Question 2

Finetune a GPT-2 model using the dataset and compare the responses of your model and the finetuned GPT-2 model. (18 marks)

In [48]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup

In [49]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def calculate_perplexity(model, dataloader, device):
    model.eval()
    total_loss = 0
    total_tokens = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Calculating perplexity"):
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)

            # Get model outputs
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss

            # Calculate number of tokens in batch (excluding padding)
            num_tokens = torch.sum(attention_mask).item()

            total_loss += loss.item() * num_tokens
            total_tokens += num_tokens

    # Calculate perplexity
    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)

    # Return both perplexity and average loss
    return perplexity, avg_loss

In [50]:
def finetune_gpt2(tokenized_train, tokenized_val, tokenized_test, test_samples, batch_size=8, epochs=100, patience=3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    set_seed(42)

    model = GPT2LMHeadModel.from_pretrained('gpt2')
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model.to(device)

    train_dataset = prepare_data_for_training(tokenized_train, seq_length=128)
    val_dataset = prepare_data_for_training(tokenized_val, seq_length=128)
    test_dataset = prepare_data_for_training(tokenized_test, seq_length=128)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

    print("\nCalculating initial perplexities...")
    initial_train_perplexity, initial_train_loss = calculate_perplexity(model, train_dataloader, device)
    initial_val_perplexity, initial_val_loss = calculate_perplexity(model, val_dataloader, device)
    initial_test_perplexity, initial_test_loss = calculate_perplexity(model, test_dataloader, device)

    print(f"Initial metrics:")
    print(f"Train - Loss: {initial_train_loss:.4f}, Perplexity: {initial_train_perplexity:.2f}")
    print(f"Val   - Loss: {initial_val_loss:.4f}, Perplexity: {initial_val_perplexity:.2f}")
    print(f"Test  - Loss: {initial_test_loss:.4f}, Perplexity: {initial_test_perplexity:.2f}")

    optimizer = AdamW(model.parameters(), lr=2e-5)
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    best_val_loss = float('inf')
    early_stopping_counter = 0
    val_losses = []

    print("\nStarting training...")
    for epoch in range(epochs):
        # Training
        model.train()
        total_train_loss = 0
        train_progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{epochs}')

        for batch in train_progress_bar:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)

            model.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss
            total_train_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            train_progress_bar.set_postfix({'loss': loss.item()})

        # Calculate epoch metrics
        print("\nCalculating epoch metrics...")
        train_perplexity, avg_train_loss = calculate_perplexity(model, train_dataloader, device)
        val_perplexity, avg_val_loss = calculate_perplexity(model, val_dataloader, device)
        test_perplexity, avg_test_loss = calculate_perplexity(model, test_dataloader, device)

        print(f"\nEpoch {epoch+1}:")
        print(f"Train - Loss: {avg_train_loss:.4f}, Perplexity: {train_perplexity:.2f}")
        print(f"Val   - Loss: {avg_val_loss:.4f}, Perplexity: {val_perplexity:.2f}")
        print(f"Test  - Loss: {avg_test_loss:.4f}, Perplexity: {test_perplexity:.2f}")

        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            early_stopping_counter = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': avg_train_loss,
                'train_perplexity': train_perplexity,
                'val_loss': avg_val_loss,
                'val_perplexity': val_perplexity,
                'test_loss': avg_test_loss,
                'test_perplexity': test_perplexity
            }, 'best_gpt2_model.pt')
            print("Saved new best model!")
        else:
            early_stopping_counter += 1
            print(f"Validation loss did not improve. Counter: {early_stopping_counter}/{patience}")

        val_losses.append(avg_val_loss)

        # Check if validation loss is continuously increasing
        if early_stopping_counter >= patience:
            print(f"\nEarly stopping triggered! Validation loss hasn't improved for {patience} epochs.")
            break

    # Load best model for generation
    print("\nLoading best model for generation...")
    checkpoint = torch.load('best_gpt2_model.pt')
    model.load_state_dict(checkpoint['model_state_dict'])

    print(f"\nBest model metrics (from epoch {checkpoint['epoch']+1}):")
    print(f"Train - Loss: {checkpoint['train_loss']:.4f}, Perplexity: {checkpoint['train_perplexity']:.2f}")
    print(f"Val   - Loss: {checkpoint['val_loss']:.4f}, Perplexity: {checkpoint['val_perplexity']:.2f}")
    print(f"Test  - Loss: {checkpoint['test_loss']:.4f}, Perplexity: {checkpoint['test_perplexity']:.2f}")

    model.eval()

    # Generate text for half of test samples
    print("\nGenerating text for test samples...")
    test_samples_subset = test_samples[:len(test_samples)//2]
    generated_outputs = []

    for sample in tqdm(test_samples_subset):
        encoded_input = tokenizer.encode(sample, return_tensors='pt').to(device)

        with torch.no_grad():
            output = model.generate(
                encoded_input,
                max_length=150,
                num_return_sequences=1,
                pad_token_id=tokenizer.eos_token_id,
                no_repeat_ngram_size=2,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=0.7
            )

        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_outputs.append(generated_text)

    # Print original and generated texts
    print("\nOriginal vs Generated Texts:")
    for original, generated in zip(test_samples_subset, generated_outputs):
        print("\nOriginal:", original)
        print("Generated:", generated)
        print("-" * 80)

    return model, generated_outputs, checkpoint

In [60]:
if __name__ == "__main__":
    model, outputs, best_checkpoint = finetune_gpt2(
        tokenized_train,
        tokenized_val,
        tokenized_test,
        test_samples,
        batch_size=8,
        epochs=1,
        patience=1  # Number of epochs to wait before early stopping
    )


Calculating initial perplexities...


Calculating perplexity: 100%|██████████| 670/670 [01:05<00:00, 10.22it/s]


Initial metrics:
Train - Loss: 7.5942, Perplexity: 1986.72
Val   - Loss: 7.5886, Perplexity: 1975.57
Test  - Loss: 7.5776, Perplexity: 1953.98

Starting training...


Epoch 1/1: 100%|██████████| 5353/5353 [30:25<00:00,  2.93it/s, loss=0.511]



Calculating epoch metrics...


Calculating perplexity: 100%|██████████| 670/670 [01:05<00:00, 10.21it/s]



Epoch 1:
Train - Loss: 0.5121, Perplexity: 1.67
Val   - Loss: 0.5383, Perplexity: 1.71
Test  - Loss: 0.5211, Perplexity: 1.68
Saved new best model!

Loading best model for generation...

Best model metrics (from epoch 1):
Train - Loss: 0.5121, Perplexity: 1.67
Val   - Loss: 0.5383, Perplexity: 1.71
Test  - Loss: 0.5211, Perplexity: 1.68

Generating text for test samples...


100%|██████████| 2/2 [00:00<00:00, 71.22it/s]


Original vs Generated Texts:

Original: whats long and hard and full of semen a submarine
Generated: whats long and hard and full of semen a submarine
--------------------------------------------------------------------------------

Original: did you hear about the knifewielding madman who attacked a circus camp the other day he went straight for the juggler
Generated: did you hear about the knifewielding madman who attacked a circus camp the other day he went straight for the juggler
--------------------------------------------------------------------------------


In [61]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [65]:
def generate_from_samples(model, tokenizer, test_samples, num_samples=5, device=None):
    """
    Generate text using a fine-tuned GPT-2 model for random test samples.
    
    Args:
        model: Fine-tuned GPT-2 model
        tokenizer: GPT-2 tokenizer
        test_samples: List of all test samples
        num_samples: Number of random samples to use
        device: Torch device (cuda/cpu)
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Set model to evaluation mode
    model.eval()
    model.to(device)
    
    # Randomly select samples
    random.seed(42)
    selected_samples = random.sample(test_samples, num_samples)
    generated_outputs = []
    
    print("\nGenerating text for test samples...")
    for sample in tqdm(selected_samples):
        encoded_input = tokenizer.encode(sample, return_tensors='pt').to(device)
        
        with torch.no_grad():
            output = model.generate(
                encoded_input,
                max_length=150,
                num_return_sequences=1,
                pad_token_id=tokenizer.eos_token_id,
                no_repeat_ngram_size=2,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=0.7
            )
        
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_outputs.append(generated_text)
    
    # Print original and generated texts
    print("\nOriginal vs Generated Texts:")
    for original, generated in zip(selected_samples, generated_outputs):
        print("\nOriginal:", original)
        print("Generated:", generated)
        print("-" * 80)
    
    return generated_outputs

# Example usage
if __name__ == "__main__":
    # Load the saved model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Initialize model and tokenizer
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    
    # Load the saved checkpoint
    checkpoint = torch.load('best_gpt2_model.pt')
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Generate text for 5 random samples
    generated_texts = generate_from_samples(
        model=model,
        tokenizer=tokenizer,
        test_samples=test_samples,  # Assuming test_samples is defined
        num_samples=5,
        device=device
    )


Generating text for test samples...


100%|██████████| 5/5 [00:00<00:00, 31.70it/s]


Original vs Generated Texts:

Original: meteorologists people who tell something wrong and still get paid
Generated: meteorologists people who tell something wrong and still get paid a little more than someone who only knows about it
--------------------------------------------------------------------------------

Original: tifu by not having a picture of my pet ready on my cake day what were you expecting to see here
Generated: tifu by not having a picture of my pet ready on my cake day what were you expecting to see here the next day
--------------------------------------------------------------------------------

Original: whats the best part about having a deaf child you can have sex as loudly as you want
Generated: whats the best part about having a deaf child you can have sex as loudly as you want
--------------------------------------------------------------------------------

Original: scottish independence
Generated: scottish independence
-------------------------------------

In [ ]:
def generate_from_samples(model, tokenizer, test_samples, num_samples=5, device=None):
    """
    Generate text using a fine-tuned GPT-2 model for random test samples.
    Uses the first half of each sample as input and generates the continuation.
    
    Args:
        model: Fine-tuned GPT-2 model
        tokenizer: GPT-2 tokenizer
        test_samples: List of all test samples
        num_samples: Number of random samples to use
        device: Torch device (cuda/cpu)
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Set model to evaluation mode
    model.eval()
    model.to(device)
    
    # Randomly select samples
    random.seed(42)
    selected_samples = random.sample(test_samples, num_samples)
    
    # Take first half of each sample
    half_samples = [sample[:len(sample)//2] for sample in selected_samples]
    generated_outputs = []
    
    print("\nGenerating text for test samples...")
    for half_sample in tqdm(half_samples):
        encoded_input = tokenizer.encode(half_sample, return_tensors='pt').to(device)
        
        with torch.no_grad():
            output = model.generate(
                encoded_input,
                max_length=150,
                num_return_sequences=1,
                pad_token_id=tokenizer.eos_token_id,
                no_repeat_ngram_size=2,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=0.7
            )
        
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_outputs.append(generated_text)
    
    # Print original, half sample, and generated texts
    print("\nOriginal vs Generated Texts:")
    for original, half, generated in zip(selected_samples, half_samples, generated_outputs):
        print("\nOriginal full text:", original)
        print("Input (first half):", half)
        print("Generated continuation:", generated)
        print("-" * 80)
    
    return generated_outputs

# Example usage
if __name__ == "__main__":
    # Load the saved model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Initialize model and tokenizer
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    
    # Load the saved checkpoint
    checkpoint = torch.load('best_gpt2_model.pt')
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Generate text for 5 random samples
    generated_texts = generate_from_samples(
        model=model,
        tokenizer=tokenizer,
        test_samples=test_samples,  # Assuming test_samples is defined
        num_samples=5,
        device=device
    )

In [ ]:
# Outputs
# Original vs Generated Texts:
# 
# Original full text: what happens when someone spills really hot coffee on you you will get burned you idiot
# Input (first half): what happens when someone spills really hot
# Generated continuation: what happens when someone spills really hot water you cant even
# --------------------------------------------------------------------------------
# 
# Original full text: what saudi funded event ended in a massive collapse on 911 hillary clintons campaign
# Input (first half): what saudi funded event ended in a massive
# Generated continuation: what saudi funded event ended in a massive earthquake i dont know what theres but a lot of people there
# --------------------------------------------------------------------------------
# 
# Original full text: the bad zoo a man walks into a zoo the only animal in the zoo is a dog its a shizhu
# Input (first half): the bad zoo a man walks into a zoo the on
# Generated continuation: the bad zoo a man walks into a zoo the onus is on him
# --------------------------------------------------------------------------------
# 
# Original full text: what do you call an angry german sauerkraut
# Input (first half): what do you call an a
# Generated continuation: what do you call an aikido fighter with no legs a kendo fighter
# --------------------------------------------------------------------------------
# 
# Original full text: there are churches for christians synagogues for jews and 911 for muslims
# Input (first half): there are churches for christians sy
# Generated continuation: there are churches for christians syrianism and communism
--------------------------------------------------------------------------------